In [ ]:
import collections
import hashlib
import inspect
import math
import os
import random
import re
import shutil
import sys
import tarfile
import time
import zipfile
from collections import defaultdict
import pandas as pd
import requests
from IPython import display
from matplotlib import pyplot as plt
from matplotlib_inline import backend_inline

In [ ]:
import numpy as np
import torch
import torchvision
from PIL import Image
from scipy.spatial import distance_matrix
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

In [ ]:
torch.__version__
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
x = torch.arange(12, dtype=torch.float32)
x.shape

torch.Size([12])

In [ ]:
X = x.reshape(3,4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

Reverse dims??? First dim = highest dimension

(2,3,4) = 2 of 3row x 4col matrices

In [ ]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [ ]:
torch.ones((2,3,4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

elements generated by randn has a mean of 0 and a standard deviation of 1

In [ ]:
torch.randn(3,4)

tensor([[-0.0069,  0.9900,  0.0920, -0.7520],
        [-1.9692,  0.4325,  0.1048, -0.8343],
        [ 0.2308, -0.4783,  0.2633, -0.5708]])

In [ ]:
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [ ]:
print(X[-1])    # Single slice refers to axis 0 (in this case, last row)
print(X[:, -1]) # Refers to axis 1 (in this case, last column)
print(X[1, 2])  # Accessing using (row, col) indices

tensor([ 8.,  9., 10., 11.])
tensor([ 3.,  7., 11.])
tensor(6.)


In [ ]:
X[:2, :] = 0
X

tensor([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 8.,  9., 10., 11.]])

In [ ]:
x

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  9., 10., 11.])

Modifying X which is a matrix based on x, also modified x!

In [ ]:
torch.exp(x)

tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 2.9810e+03, 8.1031e+03, 2.2026e+04, 5.9874e+04])

In [ ]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [ ]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [ ]:
X.sum()

tensor(66.)

In [ ]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [ ]:
a.broadcast_to((3, 2)), b.broadcast_to((3, 2))

(tensor([[0, 0],
         [1, 1],
         [2, 2]]),
 tensor([[0, 1],
         [0, 1],
         [0, 1]]))

In [ ]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [ ]:
Z = torch.zeros_like(Y)
Z

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [ ]:
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 136143200561136
id(Z): 136143200561136


In [ ]:
Z

tensor([[ 2.,  2.,  6.,  6.],
        [ 5.,  7.,  9., 11.],
        [12., 12., 12., 12.]])

In [ ]:
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

In [ ]:
data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


iloc - integer-location based indexing

for categorical missing variables, consider one-hot encoding treating NaN as a category, filling with mode of the category, or deleting rows with missing values.

In [ ]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


for numerical, consider mean, mode, median, deletion

In [ ]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


In [ ]:
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [ ]:
X.isna().sum(axis=0)

,0
age,0
workclass,963
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,966
relationship,0
race,0
sex,0


In [ ]:
print("%age of rows with missing values: ", (len(X) - len(X.dropna())) / len(X) * 100)

%age of rows with missing values:  2.499897629089718


In [ ]:
print(pd.concat((adult.variables.name, adult.variables.type), axis = 1))

              name         type
0              age      Integer
1        workclass  Categorical
2           fnlwgt      Integer
3        education  Categorical
4    education-num      Integer
5   marital-status  Categorical
6       occupation  Categorical
7     relationship  Categorical
8             race  Categorical
9              sex       Binary
10    capital-gain      Integer
11    capital-loss      Integer
12  hours-per-week      Integer
13  native-country  Categorical
14          income       Binary


### 2.3 Linear Algebra

In [2]:
import torch

In [3]:
x = torch.tensor(3.0)
y = torch.tensor(2.0)

x+y, x-y, x*y, x/y

(tensor(5.), tensor(1.), tensor(6.), tensor(1.5000))

In [4]:
x = torch.arange(3)
x

tensor([0, 1, 2])

Images are 3rd order tensors with axes corresponding to height, width, and channel. At each location, the intensities of each color channel are stacked along the channel axis.

A collection of images, then, adds another order, and becomes 4th order tensors with images indexed along the first axis.

## 2.3 Linear Algebra Exercises:

### 1. Prove that the transpose of the transpose of a matrix is the matrix itself: $(A^T)^T = A$.

In [5]:
a = torch.arange(12).reshape((3, 4))
a.shape

torch.Size([3, 4])

In [6]:
a.T.T == a

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

### 2. Given two matrices $A$ and $B$, show that sum and transposition commute: $A^T + B^T = (A + B)^T$

In [7]:
b = torch.arange(12).reshape((3, 4))
a, b

(tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]))

In [8]:
a.T + b.T, (a+b).T

(tensor([[ 0,  8, 16],
         [ 2, 10, 18],
         [ 4, 12, 20],
         [ 6, 14, 22]]),
 tensor([[ 0,  8, 16],
         [ 2, 10, 18],
         [ 4, 12, 20],
         [ 6, 14, 22]]))

In [9]:
a.T + b.T == (a + b).T

tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]])

### 3. Given any square matrix $A$, is $A + A^T$ always symmetric? Can you prove the result by using only the results of the previous two exercises?

TO PROVE: $A + A^T = (A + A^T)^T$

As we know $(A + B)^T = A^T + B^T$,

$A + A^T = A^T + (A^T)^T$

As we know $(A^T)^T = A$,

$A + A^T = A^T + A$

Hence, proved!


### 4. We defined the tensor X of shape (2, 3, 4) in this section. What is the output of len(X)? Write your answer without implementing any code, then check your answer using code.

In [10]:
x = torch.arange(24).reshape((2, 3, 4))
len(x)

2

### 5. For a tensor X of arbitrary shape, does len(X) always correspond to the length of a certain axis of X? What is that axis?

Yes, it corresponds to the axis associated with the index 0. (the first dimension)  

### 6. Run A / A.sum(axis=1) and see what happens. Can you analyze the results?

In [11]:
A = torch.arange(6).reshape(3, 2)
A

tensor([[0, 1],
        [2, 3],
        [4, 5]])

In [12]:
A.sum(axis=1)

tensor([1, 5, 9])

In [13]:
A / A.sum(axis=1)

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [14]:
A.T / A.sum(axis=1)

tensor([[0.0000, 0.4000, 0.4444],
        [1.0000, 0.6000, 0.5556]])

### 7. When traveling between two points in downtown Manhattan, what is the distance that you need to cover in terms of the coordinates, i.e., in terms of avenues and streets? Can you travel diagonally?

In [15]:
coords = torch.tensor(([0.0, 0.0], [2.0, 3.0]))
grid = torch.zeros((5, 5))

for i in coords:
    grid[int(i[0]), int(i[1])] = 1

In [16]:
# Manhattan distance

torch.abs(coords[1] - coords[0]).sum()

tensor(5.)

In [17]:
# L2 Euclidean distance

torch.norm(coords[1] - coords[0])

tensor(3.6056)

### 8. Consider a tensor of shape (2, 3, 4). What are the shapes of the summation outputs along axes 0, 1, and 2?

In [20]:
x = torch.arange(24).reshape((2, 3, 4))
x.sum(axis=0).shape, x.sum(axis=1).shape, x.sum(axis=2).shape

(torch.Size([3, 4]), torch.Size([2, 4]), torch.Size([2, 3]))

### 9. Feed a tensor with three or more axes to the linalg.norm function and observe its output. What does this function compute for tensors of arbitrary shape?

In [23]:
from scipy import linalg

In [24]:
linalg.norm

<function scipy.linalg._misc.norm(a, ord=None, axis=None, keepdims=False, check_finite=True)>

In [25]:
x = torch.arange(48).reshape((2, 2, 3, 4))
linalg.norm(x)

188.9973544788392

In [29]:
torch.sqrt(torch.sum(x**2))

tensor(188.9974)

It basically calculates the Frobenius norm